# DataTables, Indexes, Pandas

## Some useful (free) resources

Introductory:

* [Getting started with Python for research](https://github.com/TiesdeKok/LearnPythonforResearch), a gentle introduction to Python in data-intensive research.

* [A Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/index.html), by Jake VanderPlas, another quick Python intro (with notebooks).

Core Pandas/Data Science books:

* [The Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/), by Jake VanderPlas.

* [Python for Data Analysis, 2nd Edition](http://proquest.safaribooksonline.com/book/programming/python/9781491957653), by  Wes McKinney, creator of Pandas. [Companion Notebooks](https://github.com/wesm/pydata-book)

* [Effective Pandas](https://github.com/TomAugspurger/effective-pandas), a book by Tom Augspurger, core Pandas developer.


Complementary resources:

* [An introduction to "Data Science"](https://github.com/stefanv/ds_intro), a collection of Notebooks by BIDS' [Stéfan Van der Walt](https://bids.berkeley.edu/people/st%C3%A9fan-van-der-walt).

* [Effective Computation in Physics](http://proquest.safaribooksonline.com/book/physics/9781491901564), by Kathryn D. Huff; Anthony Scopatz. [Notebooks to accompany the book](https://github.com/physics-codes/seminar). Don't be fooled by the title, it's a great book on modern computational practices with very little that's physics-specific.

OK, let's load and configure some of our core libraries (as an aside, you can find a nice visual gallery of available matplotlib styles [here](https://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html)).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.options.display.max_rows = 7
pd.options.display.max_columns = 8
pd.set_option('precision', 2)
# This option stops scientific notation for pandas
# pd.set_option('display.float_format', '{:.2f}'.format)

## Reading in DataFrames from Files

Pandas has a number of very useful file reading tools. You can see them enumerated by typing "pd.re" and pressing tab. We'll be using read_csv today. 

In [2]:
elections = pd.read_csv("elections.csv")
elections # if we end a cell with an expression or variable name, the result will print

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
...,...,...,...,...,...
20,Romney,Republican,47.2,2012,loss
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


In [3]:
elections.shape

(23, 5)

In [4]:
elections.size

115

In [5]:
elections.describe()

,%,Year
count,23.00,23.00
mean,42.51,1996.87
std,13.48,11.63
...,...,...
50%,47.20,1996.00
75%,49.95,2006.00
max,58.80,2016.00


We can use the head command to show only a few rows of a dataframe.

In [6]:
elections.head(7)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss
5,Bush,Republican,53.4,1988,win
6,Dukakis,Democratic,45.6,1988,loss


There is also a tail command.

In [7]:
elections.tail(7)

,Candidate,Party,%,Year,Result
16,Bush,Republican,50.7,2004,win
17,Obama,Democratic,52.9,2008,win
18,McCain,Republican,45.7,2008,loss
19,Obama,Democratic,51.1,2012,win
20,Romney,Republican,47.2,2012,loss
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


The read_csv command lets us specify a column to use an index. For example, we could have used Year as the index.

In [8]:
elections_year_index = pd.read_csv("elections.csv", index_col = "Year")
elections_year_index.head(5)

,Candidate,Party,%,Result
Year,,,,
1980,Reagan,Republican,50.7,win
1980,Carter,Democratic,41.0,loss
1980,Anderson,Independent,6.6,loss
1984,Reagan,Republican,58.8,win
1984,Mondale,Democratic,37.6,loss


Alternately, we could have used the set_index commmand.

In [9]:
elections_party_index = elections.set_index("Party")
elections_party_index.head(5)

,Candidate,%,Year,Result
Party,,,,
Republican,Reagan,50.7,1980,win
Democratic,Carter,41.0,1980,loss
Independent,Anderson,6.6,1980,loss
Republican,Reagan,58.8,1984,win
Democratic,Mondale,37.6,1984,loss


The set_index command (along with almost all other data frame methods) does not modify the dataframe. That is, the original "elections" is untouched. Note: There is a flag called "inplace" which does modify the calling dataframe.

In [17]:
elections.head() #the index remains unchanged

,Candidate,Party,%,Result
Year,,,,
1980,Reagan,Republican,50.7,win
1980,Carter,Democratic,41.0,loss
1980,Anderson,Independent,6.6,loss
1984,Reagan,Republican,58.8,win
1984,Mondale,Democratic,37.6,loss


In [18]:
original = elections
elections = elections.set_index('Year')

original.head(2)

KeyError: "None of ['Year'] are in the columns"

In [19]:
elections.head(2)

,Candidate,Party,%,Result
Year,,,,
1980,Reagan,Republican,50.7,win
1980,Carter,Democratic,41.0,loss


In [20]:
elections = original
elections.set_index('Year', inplace=True)
original.head(2)

KeyError: "None of ['Year'] are in the columns"

In [21]:
elections = pd.read_csv('elections.csv')
elections.head(2)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss


By contast, column names MUST be unique. For example, if we try to read in a file for which column names are not unique, Pandas will automatically any duplicates.

## The [] Operator

The DataFrame class has an indexing operator [] that lets you do a variety of different things. If your provide a String to the [] operator, you get back a Series corresponding to the requested label.

In [15]:
elections["Candidate"].head(6)

0      Reagan
1      Carter
2    Anderson
3      Reagan
4     Mondale
5        Bush
Name: Candidate, dtype: object

The [] operator also accepts a list of strings. In this case, you get back a DataFrame corresponding to the requested strings.

In [16]:
elections[["Candidate", "Party"]].head(6)

,Candidate,Party
0,Reagan,Republican
1,Carter,Democratic
2,Anderson,Independent
3,Reagan,Republican
4,Mondale,Democratic
5,Bush,Republican


A list of one label also returns a DataFrame. This can be handy if you want your results as a DataFrame, not a series.

In [17]:
elections[["Candidate"]].head(6)

,Candidate
0,Reagan
1,Carter
2,Anderson
3,Reagan
4,Mondale
5,Bush


Note that we can also use the to_frame method to turn a Series into a DataFrame.

In [18]:
elections["Candidate"].to_frame()

,Candidate
0,Reagan
1,Carter
2,Anderson
...,...
20,Romney
21,Clinton
22,Trump


The [] operator also accepts numerical slices as arguments. In this case, we are indexing by row, not column!

In [22]:
elections[0:3]

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss


If you provide a single argument to the [] operator, it tries to use it as a name. This is true even if the argument passed to [] is an integer. 

In [20]:
#elections[0] #this does not work, try uncommenting this to see it fail in action, woo

The following cells allow you to test your understanding.

In [23]:
weird = pd.DataFrame({1:["topdog","botdog"], "1":["topcat","botcat"]})
weird

,1,1
0,topdog,topcat
1,botdog,botcat


In [24]:
weird[1] #try to predict the output

0    topdog
1    botdog
Name: 1, dtype: object

In [25]:
weird[["1"]] #try to predict the output

,1
0,topcat
1,botcat


In [26]:
weird[1:] #try to predict the output

,1,1
1,botdog,botcat


## Boolean Array Selection

The `[]` operator also supports array of booleans as an input. In this case, the array must be exactly as long as the number of rows. The result is a filtered version of the data frame, where only rows corresponding to True appear.

In [27]:
elections[[True, True, False, False, False, True, True, True, False, False,
           True, True, True, False, False, True, True, True, False, False,
           True, True, True]]

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
5,Bush,Republican,53.4,1988,win
...,...,...,...,...,...
20,Romney,Republican,47.2,2012,loss
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


In [28]:
elections[(elections['%'] > 50) & (elections['Party'] == 'Democratic')]
# elections['%'] > 50

,Candidate,Party,%,Year,Result
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win


In [29]:
elections[[False, False, False, False, False, 
          False, False, True, False, False,
          True, False, False, False, True,
          False, False, False, False, False,
          False, False, True]]

,Candidate,Party,%,Year,Result
7,Clinton,Democratic,43.0,1992,win
10,Clinton,Democratic,49.2,1996,win
14,Bush,Republican,47.9,2000,win
22,Trump,Republican,46.1,2016,win


One very common task in Data Science is filtering. Boolean Array Selection is one way to achieve this in Pandas. We start by observing logical operators like the equality operator can be applied to Pandas Series data to generate a Boolean Array. For example, we can compare the 'Result' column to the String 'win':

In [30]:
elections.head(5)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss


In [31]:
iswin = elections['Result'] == 'win'
iswin.head(5)

0     True
1    False
2    False
3     True
4    False
Name: Result, dtype: bool

In [28]:
elections[iswin]

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
3,Reagan,Republican,58.8,1984,win
5,Bush,Republican,53.4,1988,win
...,...,...,...,...,...
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win
22,Trump,Republican,46.1,2016,win


The output of the logical operator applied to the Series is another Series with the same name and index, but of datatype boolean. The entry with index i represents the result of the application of that operator to the entry of the original Series with index i.

In [29]:
elections[elections['Party'] == 'Independent']

,Candidate,Party,%,Year,Result
2,Anderson,Independent,6.6,1980,loss
9,Perot,Independent,18.9,1992,loss
12,Perot,Independent,8.4,1996,loss


In [30]:
elections['Result'].head(5)

0     win
1    loss
2    loss
3     win
4    loss
Name: Result, dtype: object

These boolean Series can be used as an argument to the [] operator. For example, the following code creates a DataFrame of all election winners since 1980.

In [31]:
elections.loc[iswin]

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
3,Reagan,Republican,58.8,1984,win
5,Bush,Republican,53.4,1988,win
...,...,...,...,...,...
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win
22,Trump,Republican,46.1,2016,win


Above, we've assigned the result of the logical operator to a new variable called `iswin`. This is uncommon. Usually, the series is created and used on the same line. Such code is a little tricky to read at first, but you'll get used to it quickly.

In [32]:
elections[elections['Result'] == 'win']

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
3,Reagan,Republican,58.8,1984,win
5,Bush,Republican,53.4,1988,win
...,...,...,...,...,...
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win
22,Trump,Republican,46.1,2016,win


We can select multiple criteria by creating multiple boolean Series and combining them using the `&` operator.

In [33]:
elections[(elections['Result'] == 'win')
          & (elections['%'] < 50)]

,Candidate,Party,%,Year,Result
7,Clinton,Democratic,43.0,1992,win
10,Clinton,Democratic,49.2,1996,win
14,Bush,Republican,47.9,2000,win
22,Trump,Republican,46.1,2016,win


## Loc and ILOC

In [34]:
elections.head(5)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss


In [35]:
elections.loc[[0, 1, 2, 3, 4], ['Candidate','Party', 'Year']]

,Candidate,Party,Year
0,Reagan,Republican,1980
1,Carter,Democratic,1980
2,Anderson,Independent,1980
3,Reagan,Republican,1984
4,Mondale,Democratic,1984


Loc also supports slicing (for all types, including numeric and string labels!). Note that the slicing for loc is **inclusive**, even for numeric slices.

In [36]:
elections.loc[0:4, 'Candidate':'Year']

,Candidate,Party,%,Year
0,Reagan,Republican,50.7,1980
1,Carter,Democratic,41.0,1980
2,Anderson,Independent,6.6,1980
3,Reagan,Republican,58.8,1984
4,Mondale,Democratic,37.6,1984


If we provide only a single label for the column argument, we get back a Series.

In [37]:
elections.loc[0:4, 'Candidate']

0      Reagan
1      Carter
2    Anderson
3      Reagan
4     Mondale
Name: Candidate, dtype: object

If we want a data frame instead and don't want to use to_frame, we can provde a list containing the column name.

In [38]:
elections.loc[0:4, ['Candidate']]

,Candidate
0,Reagan
1,Carter
2,Anderson
3,Reagan
4,Mondale


If we give only one row but many column labels, we'll get back a Series corresponding to a row of the table. This new Series has a neat index, where each entry is the name of the column that the data came from.

In [39]:
elections.loc[0, 'Candidate':'Year']

Candidate        Reagan
Party        Republican
%                    51
Year               1980
Name: 0, dtype: object

In [40]:
elections.loc[[0], 'Candidate':'Year']

,Candidate,Party,%,Year
0,Reagan,Republican,50.7,1980


If we omit the column argument altogether, the default behavior is to retrieve all columns. 

In [41]:
elections.loc[[2, 4, 5]]

,Candidate,Party,%,Year,Result
2,Anderson,Independent,6.6,1980,loss
4,Mondale,Democratic,37.6,1984,loss
5,Bush,Republican,53.4,1988,win


Loc also supports boolean array inputs instead of labels. If the arrays are too short, loc assumes the missing values are False.

In [42]:
elections.loc[[True, False, False, True], [True, False, False, True]]

,Candidate,Year
0,Reagan,1980
3,Reagan,1984


In [43]:
elections.loc[[0, 3], ['Candidate', 'Year']]

,Candidate,Year
0,Reagan,1980
3,Reagan,1984


We can use boolean array arguments for one axis of the data, and labels for the other.

In [44]:
elections.loc[[True, False, False, True], 'Candidate':'%']

,Candidate,Party,%
0,Reagan,Republican,50.7
3,Reagan,Republican,58.8


Boolean Series are also boolean arrays, so we can use the Boolean Array Selection from earlier using loc as well.

In [45]:
elections.loc[(elections['Result'] == 'win') & (elections['%'] < 50), 
              'Candidate':'%']

,Candidate,Party,%
7,Clinton,Democratic,43.0
10,Clinton,Democratic,49.2
14,Bush,Republican,47.9
22,Trump,Republican,46.1


Let's do a quick example using data with string-labeled rows instead of integer labeled rows, just to make sure we're really understanding loc.

As you'd expect, the rows to extract can be specified using slice notation, even if the rows have string labels instead of integer labels.

Sometimes students are so used to thinking of rows as numbered that they try the following, which will not work.

### iloc

loc's cousin iloc is very similar, but is used to access based on numerical position instead of label. For example, to access to the top 3 rows and top 3 columns of a table, we can use [0:3, 0:3]. iloc slicing is **exclusive**, just like standard Python slicing of numerical values.

In [50]:
elections.head(5)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss


In [51]:
elections.iloc[0:3, 0:3]

,Candidate,Party,%
0,Reagan,Republican,50.7
1,Carter,Democratic,41.0
2,Anderson,Independent,6.6


We will use both loc and iloc in the course. Loc is generally preferred for a number of reasons, for example: 

1. It is harder to make mistakes since you have to literally write out what you want to get.
2. Code is easier to read, because the reader doesn't have to know e.g. what column #31 represents.
3. It is robust against permutations of the data, e.g. the social security administration switches the order of two columns.

However, iloc is sometimes more convenient. We'll provide examples of when iloc is the superior choice.

## Handy Properties and Utility Functions for Series and DataFrames

The head, shape, size, and describe methods can be used to quickly get a good sense of the data we're working with. For example:

In [61]:
elections.sort_values('%')

,Candidate,Party,%,Year,Result
2,Anderson,Independent,6.6,1980,loss
12,Perot,Independent,8.4,1996,loss
9,Perot,Independent,18.9,1992,loss
...,...,...,...,...,...
17,Obama,Democratic,52.9,2008,win
5,Bush,Republican,53.4,1988,win
3,Reagan,Republican,58.8,1984,win


As mentioned before, all Data Frame methods return a copy and do **not** modify the original data structure, unless you set inplace to True.

In [62]:
elections.head(5)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss


If we want to sort in reverse order, we can set `ascending=False`.

In [63]:
elections.sort_values('%', ascending=False)

,Candidate,Party,%,Year,Result
3,Reagan,Republican,58.8,1984,win
5,Bush,Republican,53.4,1988,win
17,Obama,Democratic,52.9,2008,win
...,...,...,...,...,...
9,Perot,Independent,18.9,1992,loss
12,Perot,Independent,8.4,1996,loss
2,Anderson,Independent,6.6,1980,loss


We can also use `sort_values` on Series objects.

For Series, the `value_counts` method is often quite handy.

In [65]:
elections['Party'].value_counts()

Republican     10
Democratic     10
Independent     3
Name: Party, dtype: int64